# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w~puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}

\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}

\begin{equation}
A_xy = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
from ainvert import A_1
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext timeit
%load_ext memory_profiler
A_1 = np.array(A_1)
plt.gray()
#print(A_1)

In [ ]:
def bilinear_int(img: cv2.Mat, scalex: float, scaley: float):
    imgshape = img.shape[:2]
    newimgshape = (int(imgshape[0] * scaley), int(imgshape[1]*scalex))
    newimg = np.ndarray(newimgshape)
    for y in range(newimgshape[0]):
        for x in range(newimgshape[1]):
            A = (int(y//scaley),  int(x//scalex))
            ABCDreszta = (y/scaley-A[0], x/scalex-A[1])    
            fA = img[A[0], A[1]]
            if A[1] < imgshape[1] - 1:
                fB = img[A[0], A[1] + 1]
            else:
                fB = img[A[0], A[1] - 1]
            if A[0] < imgshape[0] - 1:
                fD = img[A[0] + 1, A[1]]
            else:
                fD = img[A[0] - 1, A[1]]
            if A[0] < imgshape[0] - 1 and A[1] < imgshape[1] - 1:
                fC = img[A[0] + 1, A[1] + 1]
            else:
                fC = img[A[0] - 1, A[1] - 1]

            matrix = np.array([[fA, fD],[fB, fC]])
            i_vec = np.ndarray((2,1))
            i_vec[0,0] = 1 - ABCDreszta[0]
            i_vec[1,0] = ABCDreszta[0]
            j_vec = np.ndarray((1,2))
            j_vec[0,0] = 1 - ABCDreszta[1]
            j_vec[0,1] = ABCDreszta[1]
            newimg[y,x] = np.dot(np.dot(j_vec, matrix), i_vec)

            

    return newimg

In [ ]:
def bicubic_int(I: cv2.Mat, scalex: float, scaley: float):
    I = I.astype('int32')
    newimgshape = (int(I.shape[0] * scaley), int(I.shape[1]*scalex))
    newimg = np.ndarray(newimgshape)
    for y in range(newimgshape[0]):
        for x in range(newimgshape[1]):
            i, j = int(y/scaley), int(x/scalex)
            if 0 < i < I.shape[0] - 2 and 0 < j < I.shape[1] - 1:
                A = I[i, j]
                Ax = (I[i+1, j] - I[i-1, j])/2
                Ay = (I[i, j+1] - I[i, j-1])/2
                Axy = ((I[i+1, j+1] - I[i-1, j]) - I[i, j-1] + I[i, j])/4
                if j < I.shape[1] - 2:
                    i, j = i, j+1
                B = I[i, j]
                Bx = (I[i+1, j] - I[i-1, j])/2
                By = (I[i, j+1] - I[i, j-1])/2
                Bxy = (I[i+1, j+1] - I[i-1, j] - I[i, j-1] + I[i, j])/4
                if i < I.shape[0] - 2:
                    i, j = i + 1, j
                C = I[i, j]
                Cx = (I[i+1, j] - I[i-1, j])/2
                Cy = (I[i, j+1] - I[i, j-1])/2
                Cxy = (I[i+1, j+1] - I[i-1, j] - I[i, j-1] + I[i, j])/4
                i, j = i, j-1
                D = I[i, j]
                Dx = (I[i+1, j] - I[i-1, j])/2
                Dy = (I[i, j+1] - I[i, j-1])/2
                Dxy = (I[i+1, j+1] - I[i-1, j] - I[i, j-1] + I[i, j])/4

                x_vec = np.array([A, B, C, D, Ax, Bx, Cx, Dx, Ay, By, Cy, Dy, Axy, Bxy, Cxy, Dxy]).T
                a = A_1@x_vec
                a = a.reshape((4, 4))
                for k in range(4):
                    for l in range(4):
                        newimg[y, x] += a[k, l]*(((x/scalex)%1)**l)*(((y/scaley)%1)**k)
                if newimg[y, x] is None:
                    print("WTF")
                newimg[y, x] = np.uint8(newimg[y, x])
    return newimg.astype('uint8')

parrot = cv2.imread('parrot.bmp', cv2.IMREAD_GRAYSCALE)
chessboard = cv2.imread('chessboard.bmp', cv2.IMREAD_GRAYSCALE)
new = bicubic_int(parrot, 1.3, 1.3)
#new = cv2.resize(chessboard, dsize=(int(chessboard.shape[0]*1.5), int(chessboard.shape[1]*1.5)), interpolation=cv2.INTER_CUBIC)
plt.imshow(new)


In [ ]:


%memit new = bicubic_int(parrot, 2, 2)
%memit new = bilinear_int(parrot, 2, 2)
%timeit new = bicubic_int(parrot, 2, 2)
%timeit new = bilinear_int(parrot, 2, 2)
